In [1]:
import requests#Connect to endpoints
import time #time script
import sys
import csv
import pandas as pd
import spotipy#authentication
import spotipy.util as util#authentication
from spotipy.oauth2 import SpotifyClientCredentials#authentication

In [2]:
#Credentials
cid = '049ade7215e54c63a2b628f3784dc407'
secret = '1ad94ff2aaec4184b8ba80416ee09bc6'
redirect_uri = 'http://google.com/'
username = 'xxx'

In [3]:
#Authentication
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [7]:
def get_artist(url):
    print(url)
    try:
        resp = requests.get(url=url,
                            headers={'Authorization': 'Bearer ' + token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
    response = resp.json()
    try:
        artist_uri = response['artists']['items'][0]['uri'].split(':')
        return artist_uri[2]
    except IndexError as error:
        return

def get_albums(url,artist_uri):
    all_albums = []
    try:
        resp = requests.get(url=url,
                            headers={'Authorization': 'Bearer ' + token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        
    response = resp.json()

    album_limit = (response['limit'])-1
    
    for x in range(album_limit):
        try:
            album_uri = response['items'][x]['uri'].split(':')
            all_albums.append(album_uri[2])
        except IndexError as error:
            continue #Natural part of the process. The length of response is sometimes less than limit
    try:
        if (url is not None):
            get_albums(response['next'])
        else:
            return
    except:
        return all_albums
    
def get_album_tracks(album_uri):
    pass
    
    

In [11]:
x = 0
artist_albums=[]

start = time.time()
with open('Data/clean_lowercase_musicDB_final.csv') as file:
    next(file)
    head = [next(file) for x in range(1)] #for limiting amount of rows read
    print(head)
    for row in head:
        #with open('Data/Spotify_Musicdb_artist_id.txt','a') as outfile:
        try:
            artist = row.replace('\n','')
            artist = row.strip()
            artist = row.replace(' ','%')
            print(artist)
            artist_uri = get_artist('https://api.spotify.com/v1/search?q={a}&type=artist'.format(a=artist))#output artist uri
            albums = get_albums('https://api.spotify.com/v1/artists/{id}/albums?'.format(id=row[2]),row[2])#list of album uris
            for album in albums:
                artist_albums.append({'Artist Name':row,'Artist URI':artist_uri,'Album URI':album})#output album_uri
                
                #album_tracks = #use https://api.spotify.com/v1/albums/{uri}/tracks: fill in id with album
                print(row,artist_uri,album)
                x+=1
        except:
            continue

stop = time.time()
duration = (stop - start)/60

['al di meola \n']
al%di%meola%

https://api.spotify.com/v1/search?q=al%di%meola%
&type=artist
400 Client Error: Bad Request for url: https://api.spotify.com/v1/artists/%20/albums


In [23]:
#Duration of script
str(duration)+' minutes'

'1212.1697368621826 minutes'

In [18]:
df = pd.DataFrame(artist_albums)
df

,artist,album
0,spotify:artist:79BoW9by11VQdVJHy6nelQ,7xbyqbmIudUFYHsgHEvcwp
1,spotify:artist:79BoW9by11VQdVJHy6nelQ,0HSSxy1xsVQNMobVl0Cr37
2,spotify:artist:79BoW9by11VQdVJHy6nelQ,1Fgb628XzkhBllD7xOZpV9
3,spotify:artist:79BoW9by11VQdVJHy6nelQ,3UzuqCIPD2JiWHoLWro8Id
4,spotify:artist:79BoW9by11VQdVJHy6nelQ,001oY5Lq7TK2VgNwRVloiD
...,...,...
34704,spotify:artist:79BoW9by11VQdVJHy6nelQ,3ioJCXzcpaDJqsVAkLQ2uB
34705,spotify:artist:79BoW9by11VQdVJHy6nelQ,6sH5Q8GrmzFu2pbjl1yXcB
34706,spotify:artist:79BoW9by11VQdVJHy6nelQ,2BvDVDifBe8W3y3p1DXHV8
34707,spotify:artist:79BoW9by11VQdVJHy6nelQ,0f0ZRGm8fvljbPZYz3Xhl0
